# Assignment 3 (Conversational AI)
> This assignment is solved by:-
* Name: Vishal Sehgal
* Group: CO11
* Roll Number: 102003265

### Question 1

In [1]:
import requests
from bs4 import BeautifulSoup

> Extracting links for the books on the first two pages

In [2]:
def get_data(pageno):
    url = "https://www.amazon.in/gp/bestsellers/books/ref=zg_bs_pg_"+str(pageno)+"?ie=UTF8&pg="+str(pageno)
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    complete = []
    for d in soup.find_all('div', attrs={'class': "p13n-sc-uncoverable-faceout"}):
        link=d.find('a',attrs={'class':"a-link-normal"})
        sub=[]
        if link is not None:
            l = 'https://www.amazon.in'+link.get('href')
            sub.append(l)
        else:
            sub.append('NaN')
        complete.append(sub)
    return complete

In [45]:
number_of_pages = 2
result = []
for i in range(1, number_of_pages):
  result.append(get_data(i))

In [48]:
import numpy as np
urls = np.array(result)
urls = urls.flatten()

> Extracting information from the every link in the list using selenium's webdriver

In [24]:
from selenium import webdriver
PATH="/Users/vishalsehgal/Documents/Programming/Web Scrapping/selenium/chromedriver"

In [50]:
complete = []
for url in urls:
    driver = webdriver.Chrome(PATH)
    driver.get(url)
    search = driver.find_element_by_id("detailBullets_feature_div")
    l = search.find_elements_by_class_name("a-list-item")
    data_dict = {}
    for i in l:
        data = i.text.split()
        col_index = 0
        for i in range(len(data)):
            if(data[i]==':'):
                col_index = i
                break
        data_dict[data[0]] = " ".join([d for d in data[col_index+1:]])
    review_search = driver.find_element_by_id("cm-cr-dp-review-list")
    reviews = review_search.find_elements_by_class_name("reviewText")
    review = []
    itr = 0
    for i in reviews:
        if(itr==3):
            break
        review.append(i.text)
        itr+=1
    sub = []
    # i.
    if('Publisher' in data_dict):
        if(data_dict['Publisher'] is not None):
            sub.append(data_dict['Publisher'])
        else:
            sub.append('NaN')
    else:
        sub.append('NaN')
    # ii.
    if('Language' in data_dict):
        if(data_dict['Language'] is not None):
            sub.append(data_dict['Language'])
        else:
            sub.append('NaN')
    else:
        sub.append('NaN')
    # iii.
    if('Paperback' in data_dict):
        if(data_dict['Paperback'] is not None):
            sub.append(data_dict['Paperback'])
        else:
            sub.append('NaN')
    else:
        sub.append('NaN')
    # iv.
    if('ISBN-10' in data_dict):
        if(data_dict['ISBN-10'] is not None):
            sub.append(data_dict['ISBN-10'])
        else:
            sub.append('NaN')
    else:
        sub.append('NaN')
    # v.
    if('ISBN-13' in data_dict):
        if(data_dict['ISBN-13'] is not None):
            sub.append(data_dict['ISBN-13'])
        else:
            sub.append('NaN')
    else:
        sub.append('NaN')
    # vi.
    if('Item' in data_dict):
        if(data_dict['Item'] is not None):
            sub.append(data_dict['Item'])
        else:
            sub.append('NaN')
    else:
        sub.append('NaN')
    # vii.
    if('Dimensions' in data_dict):
        if(data_dict['Dimensions'] is not None):
            sub.append(data_dict['Dimensions'])
        else:
            sub.append('NaN')
    else:
        sub.append('NaN')
    # viii.
    if(review is not None):
        sub.append(review)
    else:
        sub.append('NaN')
    complete.append(sub)
    driver.quit()

> Converting information into a DataFrame

In [53]:
import pandas as pd
df = pd.DataFrame(complete, columns=['Publisher', 'Language', 'Paperback', 'ISBN-10', 'ISBN-13', 'Weight', 'Dimensions', 'Reviews (First Three)'])
df.head()

,Publisher,Language,Paperback,ISBN-10,ISBN-13,Weight,Dimensions,Reviews (First Three)
0,Simon & Schuster (18 October 2022); Simon & Sc...,English,352 pages,1398518174,978-1398518179,370 g,15.2 x 2.3 x 22.9 cm,[I was waiting since I had read... It End With...
1,HarperCollins India (3 October 2022),English,500 pages,9356291527,978-9356291522,370 g,20.3 x 25.4 x 4.7 cm,[The fourth book of the Ram Chandra series has...
2,Random House Business Books (30 October 2018),English,288 pages,1847941834,978-1847941831,419 g,15.3 x 2.3 x 23.4 cm,[1. Don't set Goals : Writer kehte he ki goal ...
3,Random House UK (27 September 2017),English,NaN,178633089X,978-1786330895,247 g,13.6 x 2.3 x 18.4 cm,[It is a very good book to read and go through...
4,Jaico Publishing House (1 September 2020); Jai...,English,252 pages,9390166268,978-9390166268,220 g,20.3 x 25.4 x 4.7 cm,[It has all necessary and fundamental thought ...


> Saving information as .csv file

In [56]:
df.to_csv('amazon_top_books.csv', index=False)

### Question 2

In [76]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
PATH = "/Users/vishalsehgal/Documents/Programming/Web Scrapping/selenium/chromedriver"

In [77]:
import numpy as np
number_of_movies = 1000
arr = np.arange(1, number_of_movies, 50)

In [78]:
complete = []
for ele in arr:
    url = "https://www.imdb.com/search/title/?release_date=2020-01-01,2020-12-31&sort=num_votes,desc&start="+str(ele)+"&ref_=adv_prv"
    driver = webdriver.Chrome(PATH)
    driver.get(url)
    divs = driver.find_elements_by_class_name('mode-advanced')
    for div in divs:
        sub = []
        h = div.find_element_by_class_name('lister-item-header')
        # i.
        movie_name = None
        try:
            movie_name = h.find_element_by_tag_name('a')
        except NoSuchElementException:
            movie_name = None
        if movie_name is not None:
            sub.append(movie_name.text)
        else:
            sub.append(movie_name)
        # ii.
        movie_rating = None
        try:
            movie_rating = div.find_element_by_class_name('ratings-imdb-rating')
        except NoSuchElementException:
            movie_rating = None
        if movie_rating is not None:
            sub.append(movie_rating.text)
        else:
            sub.append(movie_rating)
        # iii.
        description = None
        try:
            description = div.find_elements_by_class_name('text-muted')
            description = description[2]
        except NoSuchElementException:
            description = None
        if description is not None:
            sub.append(description.text)
        else:
            sub.append(description)
        # iv.
        votes = None
        try:
            votes = div.find_element_by_name('nv')
        except NoSuchElementException:
            votes = None
        if votes is not None:
            sub.append(votes.text)
        else:
            sub.append(votes)
        # v. & vi.
        directors_and_stars = None
        try:
            directors_and_stars = div.find_elements_by_tag_name('p')
            directors_and_stars = directors_and_stars[2]
        except NoSuchElementException:
            directors_and_stars = None
        comma = []
        if directors_and_stars is not None:
            s = directors_and_stars.text
            spt = s.split()
            directors = []
            if("Director:" in spt or "Directors:" in spt):
                start = 0
                comma = []
                for i in range(len(s)):
                    if(s[i]==":"):
                        start = i+2
                    if(s[i]==","):
                        comma.append(i)
                    if(s[i]=="|"):
                        comma.append(i-1)
                        break
                itr = 0
                for i in comma:
                    dir = []
                    if(itr != 0):
                        start = comma[itr-1]+2
                    for j in range(start, i):
                        dir.append(s[j])
                    dir = "".join([d for d in dir])
                    directors.append(dir)
                    itr+=1
            else:
                directors = None
            stars = []
            if("Stars:" in spt or "Star:" in spt):
                if(len(comma)!=0):
                    h = comma[-1]+3
                else:
                    h = 0
                comma = []
                for i in range(h, len(s)):
                    if(s[i]==":"):
                        start = i+2
                    if(s[i]==","):
                        comma.append(i)
                comma.append(len(s))
                itr = 0
                for i in comma:
                    dir = []
                    if(itr != 0):
                        start = comma[itr-1]+2
                    for j in range(start, i):
                        dir.append(s[j])
                    dir = "".join([d for d in dir])
                    stars.append(dir)
                    itr+=1
            else:
                stars = None
            sub.append(directors)
            sub.append(stars)
        else:
            sub.append(directors_and_stars)
        # vii.
        metascore = None
        try:
            metascore = div.find_element_by_class_name('metascore')
        except NoSuchElementException:
            metascore = None
        if metascore is not None:
            sub.append(metascore.text)
        else:
            sub.append(metascore)
        # viii.
        genre = None
        try:
            genre = div.find_element_by_class_name('genre')
        except NoSuchElementException:
            genre = None
        if genre is not None:
            sub.append(genre.text)
        else:
            sub.append(genre)
        # ix.
        total_duration = None
        try:
            total_duration = div.find_element_by_class_name('runtime')
        except NoSuchElementException:
            total_duration = None
        if total_duration is not None:
            sub.append(total_duration.text)
        else:
            sub.append(total_duration)
        complete.append(sub)
    driver.quit()

In [79]:
complete

[['Tenet',
  '7.3',
  'Armed with only one word, Tenet, and fighting for the survival of the entire world, a Protagonist journeys through a twilight world of international espionage on a mission that will unfold in something beyond real time.',
  '496,092',
  ['Christopher Nolan'],
  ['John David Washington',
   'Robert Pattinson',
   'Elizabeth Debicki',
   'Juhan Ulfsak'],
  '69',
  'Action, Sci-Fi, Thriller',
  '150 min'],
 ["The Queen's Gambit",
  '8.6',
  'Orphaned at the tender age of nine, prodigious introvert Beth Harmon discovers and masters the game of chess in 1960s USA. But child stardom comes at a price.',
  '439,068',
  None,
  ['Anya Taylor-Joy', 'Chloe Pirrie', 'Bill Camp', 'Marcin Dorocinski'],
  None,
  'Drama',
  '60 min'],
 ['Soul',
  '8.0',
  'After landing the gig of a lifetime, a New York jazz pianist suddenly finds himself trapped in a strange land between Earth and the afterlife.',
  '325,652',
  ['Pete Docter', 'Kemp Powers'],
  ['Jamie Foxx', 'Tina Fey', 'Gra

In [80]:
import pandas as pd
df = pd.DataFrame(complete, columns=['Movie Name', 'IMDB Rating', 'Description', 'Votes', 'Director(s)', 'Stars', 'Metascore', 'Movie Genre', 'Total Duration'])
df.head()

,Movie Name,IMDB Rating,Description,Votes,Director(s),Stars,Metascore,Movie Genre,Total Duration
0,Tenet,7.3,"Armed with only one word, Tenet, and fighting ...","496,092",[Christopher Nolan],"[John David Washington, Robert Pattinson, Eliz...",69,"Action, Sci-Fi, Thriller",150 min
1,The Queen's Gambit,8.6,"Orphaned at the tender age of nine, prodigious...","439,068",None,"[Anya Taylor-Joy, Chloe Pirrie, Bill Camp, Mar...",None,Drama,60 min
2,Soul,8.0,"After landing the gig of a lifetime, a New Yor...","325,652","[Pete Docter, Kemp Powers]","[Jamie Foxx, Tina Fey, Graham Norton, Rachel H...",83,"Animation, Adventure, Comedy",100 min
3,Wonder Woman 1984,5.4,"Diana must contend with a work colleague, and ...","265,258",[Patty Jenkins],"[Gal Gadot, Chris Pine, Kristen Wiig, Pedro Pa...",60,"Action, Adventure, Fantasy",151 min
4,Birds of Prey,6.0,"After splitting with the Joker, Harley Quinn j...","237,608",[Cathy Yan],"[Margot Robbie, Rosie Perez, Mary Elizabeth Wi...",60,"Action, Adventure, Comedy",109 min


In [81]:
df.to_csv('IMDB_movies_web_scrapped.csv', index=False)